For this project, I first tried using AI, and then switched to modifying the code of another person. The original code can be found at the following URL: https://www.kaggle.com/code/vijayaadithyanvg/car-price-prediction-used-cars

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import metrics

In [24]:
car_df = pd.read_csv('cars.csv')
# Extract brand from name column
car_df.insert(0, "make", car_df['name'].apply(lambda x: x.split()[0]))
car_df.insert(1, "model", car_df['name'].apply(lambda x: " ".join(x.split()[1:])))
car_df = car_df.drop(columns='name', axis=1)
car_df['selling_price'] = car_df['selling_price'].multiply(0.012)

car_df.sort_values(by='selling_price', ascending=False)

# car_df

,make,model,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
170,Volvo,XC90 T8 Excellence BSIV,2017,120000.000,30000,Petrol,Individual,Automatic,First Owner,42.0 kmpl,1969 CC,400 bhp,640Nm@ 1740rpm,4.0
2938,BMW,X7 xDrive 30d DPE,2020,86400.000,5000,Diesel,Individual,Automatic,First Owner,13.38 kmpl,2993 CC,265 bhp,620Nm@ 1500-2500rpm,7.0
4952,Audi,A6 35 TFSI Matrix,2019,78276.000,23600,Petrol,Dealer,Automatic,Test Drive Car,15.26 kmpl,1798 CC,187.74 bhp,320Nm@ 1400-4100rpm,5.0
4950,Audi,A6 35 TFSI Matrix,2019,74676.000,7800,Petrol,Dealer,Automatic,Test Drive Car,15.26 kmpl,1798 CC,187.74 bhp,320Nm@ 1400-4100rpm,5.0
7713,Mercedes-Benz,S-Class S 350 CDI,2017,72000.000,37000,Diesel,Dealer,Automatic,First Owner,13.5 kmpl,2987 CC,254.79 bhp,620Nm@ 1600-2400rpm,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,Maruti,800 Std,2004,378.048,110000,Petrol,Individual,Manual,Third Owner,16.1 kmpl,796 CC,37 bhp,59Nm@ 2500rpm,4.0
5965,Maruti,800 Std,2000,372.000,56194,Petrol,Individual,Manual,Fourth & Above Owner,16.1 kmpl,796 CC,37 bhp,59Nm@ 2500rpm,4.0
6776,Hyundai,Santro GLS I - Euro I,2000,360.000,90000,Petrol,Individual,Manual,Third Owner,NaN,NaN,NaN,NaN,NaN
2145,Maruti,Zen LXI,2001,360.000,10000,Petrol,Individual,Manual,First Owner,17.3 kmpl,993 CC,60 bhp,78Nm@ 4500rpm,5.0


In [25]:
car_df.shape
#rows, then columns

(8128, 14)

In [26]:
car_df.isnull().sum()

make               0
model              0
year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          221
engine           221
max_power        215
torque           222
seats            221
dtype: int64

For this analysis, I don't really care about the columns with missing data, so I don't need to drop those records.

In [27]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   make           8128 non-null   object 
 1   model          8128 non-null   object 
 2   year           8128 non-null   int64  
 3   selling_price  8128 non-null   float64
 4   km_driven      8128 non-null   int64  
 5   fuel           8128 non-null   object 
 6   seller_type    8128 non-null   object 
 7   transmission   8128 non-null   object 
 8   owner          8128 non-null   object 
 9   mileage        7907 non-null   object 
 10  engine         7907 non-null   object 
 11  max_power      7913 non-null   object 
 12  torque         7906 non-null   object 
 13  seats          7907 non-null   float64
dtypes: float64(2), int64(2), object(10)
memory usage: 889.1+ KB


In [28]:
print(car_df.selling_price.value_counts())

selling_price
3600.000    227
7200.000    213
4200.000    209
6600.000    204
5400.000    195
           ... 
2435.988      1
6492.000      1
3048.000      1
6336.000      1
1224.000      1
Name: count, Length: 677, dtype: int64
